In [1]:
#Libraries

#pip install folium
#pip install geopy
#pip install WeasyPrint
#pip install imgkit
#pip install pdfkit


import pandas as pd

from geopy.geocoders import Nominatim

import folium
from folium.plugins import MarkerCluster
import random
import string

#FUNCTIONS FOR PROJECT LOCATIONS
def check_project_coordinates(row):
    # Check if 'Project Latitude' and 'Project Longitude' are available
    if not pd.isna(row['Project Latitude']) and not pd.isna(row['Project Longitude']):
        return row['Project Latitude'], row['Project Longitude']
    return None, None

def check_location_coordinates(row):
    # Check if 'Location Latitude' and 'Location Longitude' are available
    if not pd.isna(row['Location Latitude']) and not pd.isna(row['Location Longitude']):
        return row['Location Latitude'], row['Location Longitude']
    return None, None

def geocode_location(location):
    try:
        location_info = geolocator.geocode(location)
        if location_info:
            return location_info.latitude, location_info.longitude
        else:
            print(f"Geocoding failed for location: {location}")
    except Exception as e:
        print(f"Error geocoding location '{location}': {str(e)}")
    return None, None

#1.1 - Project Locations
#CREATES FILE TO STORE LOCATIONS AND THEIR COORDS

# Initialize a geolocator using Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Read the Excel file
df = pd.read_excel("FibreLocationMappingPython.xlsx")

# List to store processed data
output_data = []

# Iterate through each row
for index, row in df.iterrows():
    # Check if the 'Project Number' is not just "0"
    if str(row['Project Number']).strip() != "0":
        project_lat, project_long = check_project_coordinates(row)
        if project_lat is not None and project_long is not None:
            latitude, longitude = project_lat, project_long
        else:
            location_lat, location_long = check_location_coordinates(row)
            if location_lat is not None and location_long is not None:
                latitude, longitude = location_lat, location_long
            else:
                latitude, longitude = geocode_location(f"{row['Project Location']}, {row['Province']}")

        output_data.append({
            'Inspector Assigned': row['Inspector Assigned'],
            'Contractor': row['Contractor'],
            'DM': row['DM'],
            'Project Number': row['Project Number'],
            'Project Location': row['Project Location'],
            'Province': row['Province'],
            'Latitude': latitude,
            'Longitude': longitude
        })

# Create a DataFrame from the output data
df_output = pd.DataFrame(output_data)

# Save the DataFrame to a CSV file
df_output.to_csv('location_coordinates.csv', index=False)

# Print path to the location coordinates CSV file
print("Path to location coordinates CSV file: location_coordinates.csv")

#FUNCTIONS FOR INSPECTOR LOCATIONS
def check_inspector_coordinates(row):
    # Check if 'Inspector Latitude' and 'Inspector Longitude' are available
    if not pd.isna(row['Inspector Latitude']) and not pd.isna(row['Inspector Longitude']):
        return row['Inspector Latitude'], row['Inspector Longitude']
    else:
        # If coordinates are not available, geocode the location
        location = f"{row['Inspector Location']}, {row['Inspector Province']}"
        try:
            location_info = geolocator.geocode(location)
            if location_info:
                return location_info.latitude, location_info.longitude
            else:
                print(f"Geocoding failed for location: {location}")
        except Exception as e:
            print(f"Error geocoding location '{location}': {str(e)}")
    return None, None

# Section 1.2 - Inspector Locations
#CREATES FILE TO STORE LOCATIONS AND THEIR COORDS

## Initialize a geolocator using Nominatim
geolocator = Nominatim(user_agent="my_geocoder")

# Read the Excel file for inspector locations
inspector_df = pd.read_excel("FibreLocationMappingPython.xlsx", sheet_name="Inspector Location")

# Filter out rows with NaN values in 'Inspector Location' and 'Inspector Province' columns
inspector_df = inspector_df.dropna(subset=['Inspector Location', 'Inspector Province'])

# List to store processed data
inspector_output_data = []

# Iterate through each row
for index, row in inspector_df.iterrows():
    inspector_lat, inspector_long = check_inspector_coordinates(row)
    if inspector_lat is not None and inspector_long is not None:
        latitude, longitude = inspector_lat, inspector_long
    else:
        # If inspector coordinates not found, geocode the location
        location = f"{row['Inspector Location']}, {row['Inspector Province']}"
        try:
            location_info = geolocator.geocode(location)
            if location_info:
                latitude, longitude = location_info.latitude, location_info.longitude
            else:
                print(f"Geocoding failed for location: {location}")
                latitude, longitude = None, None
        except Exception as e:
            print(f"Error geocoding location '{location}': {str(e)}")
            latitude, longitude = None, None

    inspector_output_data.append({
        'Inspector Name': row['Inspector Name'],
        'Inspector Location': row['Inspector Location'],
        'Inspector Province': row['Inspector Province'],
        'Latitude': latitude,
        'Longitude': longitude
    })

# Create a DataFrame from the output data
df_inspector_output = pd.DataFrame(inspector_output_data)

# Save the DataFrame to a CSV file
df_inspector_output.to_csv('inspector_coordinates.csv', index=False)

# Print path to the inspector coordinates CSV file
print(df_inspector_output)
print("\nPath to inspector coordinates CSV file: inspector_coordinates.csv")

#GOOD ONE

#FUNCTIONS FOR MAPPING
def read_coordinates_from_csv(file_path):
    """Reads location coordinates from a CSV file."""
    return pd.read_csv(file_path)

def create_map():
    """Creates a map centered around Southwest Ontario."""
    return folium.Map(location=[42.9837, -81.2497], zoom_start=8)

def assign_random_colors(inspector_list):
    """Assigns random colors to each inspector."""
    # Define a list of predefined colors in Folium excluding red
    folium_colors = ['black', 'lightblue', 'beige', 'green', 
                     'cadetblue', 'darkblue', 'darkgreen', 'pink', 'gray',
                     'lightgray', 'white', 'orange', 'blue', 'lightgreen'] #can probably add more colors
    
    # Shuffle the inspector list to ensure randomness
    random.shuffle(inspector_list)
    
    # Assign colors from the predefined list to each inspector
    inspector_colors = {}
    for i, inspector in enumerate(inspector_list):
        inspector_colors[inspector] = folium_colors[i % len(folium_colors)]
    
    return inspector_colors

def add_project_markers(map_obj, df_coordinates, df_inspector_coordinates, inspector_colors):
    """Adds markers for project locations to the map."""
    # Filter out rows with NaN values in latitude and longitude columns
    df_coordinates = df_coordinates.dropna(subset=['Latitude', 'Longitude'])

    # Add markers for each project location from the CSV file
    for index, row in df_coordinates.iterrows():
        location = row['Project Location']
        inspector_assigned = row['Inspector Assigned']
        project_number = row['Project Number']
        dm = row['DM']
        contractor = row['Contractor']

        # Initialize the popup text
        popup_text = f"Location: {location}<br>"

        # Add project number to the popup text
        popup_text += f"<b>Project Numbers:</b><br>{project_number}<br>"

        # Add inspector information to the popup text
        if pd.notna(inspector_assigned):
            popup_text += f"<b>Inspectors:</b><br>{inspector_assigned}<br>"

        # Add DM information to the popup text
        if pd.notna(dm):
            popup_text += f"<b>DM:</b><br>{dm}<br>"

        # Add contractor information to the popup text
        if pd.notna(contractor):
            popup_text += f"<b>Contractor:</b><br>{contractor}<br>"

        # Check if an inspector is assigned to this project location
        if pd.notna(inspector_assigned):
            # Check if the assigned inspector is from the list
            if inspector_assigned in inspector_colors:
                # Use the assigned inspector's color
                marker_color = inspector_colors[inspector_assigned]
            else:
                # Assign a different color if the inspector is not from the list
                marker_color = 'purple'

            # Add click events to markers for project locations
            folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup_text, icon=folium.Icon(color=marker_color, icon='map-marker')).add_to(map_obj).add_child(folium.ClickForMarker(popup=popup_text)).add_child(folium.Tooltip(text=location))

def add_inspector_markers(map_obj, df_inspector_coordinates, inspector_colors):
    """Adds markers for inspector locations to the map."""
    # Add tooltips to markers for inspector locations
    for index, row in df_inspector_coordinates.iterrows():
        location = row['Inspector Location']
        latitude = row['Latitude']
        longitude = row['Longitude']
        inspector_name = row['Inspector Name']

        # Get the color for this inspector
        inspector_color = inspector_colors.get(inspector_name, 'green')  # Default color is green

        popup_text = f"Inspector: {inspector_name}<br>Location: {location}<br>Coordinates: ({latitude}, {longitude})" # CUSTOMIZE INSPECTOR ICON

        # Add tooltips to markers for inspector locations
        folium.Marker(location=[latitude, longitude], popup=popup_text, icon=folium.Icon(color=inspector_color, icon='star')).add_to(map_obj).add_child(folium.Tooltip(text=inspector_name))

def save_map(map_obj, file_path):
    """Saves the map to an HTML file."""
    map_obj.fit_bounds(map_obj.get_bounds())
    map_obj.save(file_path)
    print(f"Map saved to {file_path}")

# Read the project location coordinates from the CSV file
df_coordinates = read_coordinates_from_csv("location_coordinates.csv")

# Read the inspector location coordinates from the CSV file
df_inspector_coordinates = read_coordinates_from_csv("inspector_coordinates.csv")

# Extract the list of inspectors
inspector_list = df_inspector_coordinates['Inspector Name'].tolist()

# Assign random colors to each inspector
inspector_colors = assign_random_colors(inspector_list)

# Create a map
map_sw_ontario = create_map()

# Add project markers to the map
add_project_markers(map_sw_ontario, df_coordinates, df_inspector_coordinates, inspector_colors)

# Add inspector markers to the map
add_inspector_markers(map_sw_ontario, df_inspector_coordinates, inspector_colors)

# Save the map to an HTML file
save_map(map_sw_ontario, "southwest_ontario_map.html")



C:\Users\efrancis\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Path to location coordinates CSV file: location_coordinates.csv
     Inspector Name                                Inspector Location  \
0   Ferzine Rasheed    50 Witteveen Dr, Brantford, ON N3T 5L5, Canada   
1   Jeffrey Sandney                   3 Moir Cres, Barrie, ON L4N 8B7   
2  Jonathan Alverez                     7895 Jane St, Concord, L4K2M7   
3   Manu Peringelil   312 Bluevale street North, Waterloo, N2J4G3, ON   
4     Nayeem Afridi            4601 Dunedin Crescent, Mississauga, ON   
5  Randy Sutherland             908 Code Dr, Smiths Falls, ON K7A 4S6   
6        Watwa Cham  802-1425B Block Line Road, Kitchener, ON N2C 0C4   
7      Edwin Lasala                                   Pearson Airport   

  Inspector Province   Latitude  Longitude  
0            Ontario  43.183041 -80.255866  
1            Ontario  44.358814 -79.661681  
2            Ontario  43.792901 -79.534846  
3            Ontario  43.466898 -80.526677  
4            Ontario  43.599204 -79.712537  
5       